# Setup

In [1]:
%matplotlib ipympl
import matplotlib, matplotlib.pyplot

matplotlib.pyplot.ioff()

from package.apps import Apps

# Queries

## Apps in prod using JsonObjectInDataResponse

In [2]:
with Apps.init() as apps:
    apps.where(
        lambda app: app.env == "prod"
        and app.app_settings.some(
            lambda app_settings: app_settings.environment in ["Production", "default"]
            and app_settings[".FeatureManagement.JsonObjectInDataResponse"] == True
        )
    ).table()

Env,Org,App
prod,digdir,tilskudd-dig-delt-komp
prod,brg,stottetildeling-ny
prod,dihe,omsetningsoppgave-for-alkohol
prod,dihe,soknad-bevillinger
prod,krt,krt-1115a-1
prod,srf,bsk-rapport
prod,srf,drk-rapport
prod,srf,ettersendelse-dokumentasjon
prod,srf,fullmakt-rettferdsvederlag
prod,srf,fullstendighet-aarlig


Count: 16


## Apps in prod using RemoveHiddenData

In [3]:
with Apps.init() as apps:
    apps.where(
        lambda app: app.env == "prod"
        and app.app_settings.some(
            lambda app_settings: app_settings.environment in ["Production", "default"]
            and app_settings[".AppSettings.RemoveHiddenData"] == True
        )
    ).table()

Env,Org,App
prod,dat,bemanning-aarligmelding
prod,dat,bemanning-varseltilbakekalling
prod,dihe,soknad-bevillinger
prod,mat,fisk-meldepliktige-hendelser
prod,lt,digital-compliance
prod,lt,drone-flight-oslo-city-center
prod,staf,avlopsvann-akutt-utslipp
prod,staf,ktu-tilskudd-soknad
prod,staf,melding-bedrift
prod,tad,aeo


Count: 71


## Apps in prod using RequiredValidation

In [4]:
with Apps.init() as apps:
    apps.where(
        lambda app: app.env == "prod"
        and app.app_settings.some(
            lambda app_settings: app_settings.environment in ["Production", "default"]
            and app_settings[".AppSettings.RequiredValidation"] == True
        )
    ).table()

Env,Org,App
prod,mat,fisk-meldepliktige-hendelser
prod,lt,digital-compliance
prod,lt,drone-flight-oslo-city-center
prod,lt,noise-complaint
prod,lt,notice-of-concern
prod,lt,objectionable-conditions
prod,staf,kopi-av-bevilling-sepskil
prod,staf,melding-bedrift
prod,staf,politikontakt-barn
prod,tad,aeo


Count: 36
